In [1]:
!pip install pandas --upgrade
import numpy as np
import pandas as pd

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.23.4)


Make sure to upload the following files:
- PC_Export_2014_2015.csv  
- PC_Export_2017_2018.csv
- PC_Import_2016_2017.csv
- PC_Export_2015_2016.csv
- PC_Import_2014_2015.csv 
- PC_Import_2017_2018.csv
- PC_Export_2016_2017.csv 
- PC_Import_2015_2016.csv 

In [2]:
#Make sure to upload the files
!ls

joined_df.csv		 PC_Export_2017_2018.csv  PC_Import_2017_2018.csv
PC_Export_2014_2015.csv  PC_Import_2014_2015.csv  processed_data_out_pd.csv
PC_Export_2015_2016.csv  PC_Import_2015_2016.csv  sample_data
PC_Export_2016_2017.csv  PC_Import_2016_2017.csv


In [0]:
years=['2014_2015','2015_2016','2016_2017','2017_2018']
df_imp_exp={'export':'PC_Export_','import':'PC_Import_'}
df_dict=dict()
for key,value in df_imp_exp.items():
  df_dict[key]=dict()
  for year in years:
    df_dict[key][year]=pd.read_csv('{}{}.csv'.format(value,year))  

In [4]:
df_dict['import']['2017_2018'].head(),df_dict['export']['2017_2018'].head()

(  PC_DESC UNIT OF QTY    COUNTRY        QTY  VAL (MILL US $)
 0     TEA         KGS  ARGENTINA  1617240.0             2.58
 1     TEA         KGS  AUSTRALIA       45.0             0.00
 2     TEA         KGS    AUSTRIA     1382.0             0.00
 3     TEA         KGS    BELGIUM       77.0             0.00
 4     TEA         KGS     CANADA      380.0             0.02,
   COMMODITY UNIT_CODE          ...                QTY  VALUES (US$ million)
 0       TEA       KGS          ...           996994.0                  2.88
 1       TEA       KGS          ...           250065.0                  0.82
 2       TEA       KGS          ...           109640.0                  0.21
 3       TEA       KGS          ...            32692.0                  0.15
 4       TEA       KGS          ...            11361.0                  0.07
 
 [5 rows x 5 columns])

In [5]:
df_dict['import']['2016_2017'].columns,df_dict['export']['2016_2017'].columns,df_dict['import']['2017_2018'].columns,df_dict['export']['2017_2018'].columns

(Index(['pc_code', 'pc_description', 'unit', 'country_code', 'country_name',
        'quantity', 'value'],
       dtype='object'),
 Index(['pc_code', 'pc_description', 'unit', 'country_code', 'country_name',
        'quantity', 'value'],
       dtype='object'),
 Index(['PC_DESC', 'UNIT OF QTY', 'COUNTRY', 'QTY', 'VAL (MILL US $)'], dtype='object'),
 Index(['COMMODITY', 'UNIT_CODE', 'COUNTRY', 'QTY', 'VALUES (US$ million)'], dtype='object'))

In [6]:
cols_rename_2017_2018={'export':{'COMMODITY':'pc_description','UNIT_CODE':'unit','COUNTRY':'country_name',
               'QTY':'quantity','VALUES (US$ million)':'value'},'import':{'PC_DESC':'pc_description',
              'UNIT OF QTY':'unit','COUNTRY':'country_name','QTY':'quantity','VAL (MILL US $)':'value'}}
for key,col_rename in cols_rename_2017_2018.items():
  df_dict[key]['2017_2018']=df_dict[key]['2017_2018'].rename(columns=col_rename)
df_dict['import']['2017_2018'].columns,df_dict['export']['2017_2018'].columns

(Index(['pc_description', 'unit', 'country_name', 'quantity', 'value'], dtype='object'),
 Index(['pc_description', 'unit', 'country_name', 'quantity', 'value'], dtype='object'))

In [7]:
df_dict['import']['2016_2017'].columns,df_dict['export']['2016_2017'].columns,df_dict['import']['2017_2018'].columns,df_dict['export']['2017_2018'].columns

(Index(['pc_code', 'pc_description', 'unit', 'country_code', 'country_name',
        'quantity', 'value'],
       dtype='object'),
 Index(['pc_code', 'pc_description', 'unit', 'country_code', 'country_name',
        'quantity', 'value'],
       dtype='object'),
 Index(['pc_description', 'unit', 'country_name', 'quantity', 'value'], dtype='object'),
 Index(['pc_description', 'unit', 'country_name', 'quantity', 'value'], dtype='object'))

In [0]:
#adding type and year as columns
for key,value in df_imp_exp.items():
  for year in years:
    df_dict[key][year]['type']=key
    df_dict[key][year]['year']=year
    df_dict[key][year]=df_dict[key][year].applymap(lambda x:x.lower() if type(x) is str else x)

In [9]:
#Checking whether the pc_description of 2017_2018 matches with those of other years
df_dict['import']['2017_2018'].pc_description.isin(df_dict['import']['2016_2017'].pc_description).sum()/df_dict['import']['2017_2018'].shape[0],df_dict['export']['2017_2018'].pc_description.isin(df_dict['export']['2016_2017'].pc_description).sum()/df_dict['export']['2017_2018'].shape[0]

(1.0, 0.9999536629442565)

In [10]:
#Deleting these 2 columns since they are not present in 2017_2018 df and they both dont add any extra info anyways
cols_to_remove=['pc_code','country_code']
mod_years=[year for year in years if year!='2017_2018']
for key in df_imp_exp.keys():
  for year in mod_years:
    df_dict[key][year]=df_dict[key][year].drop(cols_to_remove,axis=1)
df_dict['import']['2016_2017'].columns

Index(['pc_description', 'unit', 'country_name', 'quantity', 'value', 'type',
       'year'],
      dtype='object')

In [0]:
#Sanity Check...checking whether the columns for all the individual dfs are the same
cols=list(df_dict['import']['2016_2017'].columns)
for key in df_imp_exp.keys():
  for year in years:
    assert list(df_dict[key][year].columns)==cols,"Check df_dict[{}][{}]".format(key,year)

In [12]:
dfs_list=[]
for key in df_imp_exp.keys():
  for year in years:
    dfs_list.append(df_dict[key][year])
joined_df=pd.concat(dfs_list,ignore_index=True)
joined_df.shape,joined_df.head()

((122939, 7),       pc_description unit    ...        type       year
 0  other commodities   na    ...      export  2014_2015
 1  other commodities   na    ...      export  2014_2015
 2  other commodities   na    ...      export  2014_2015
 3  other commodities   na    ...      export  2014_2015
 4  other commodities   na    ...      export  2014_2015
 
 [5 rows x 7 columns])

In [13]:
(joined_df.loc[(joined_df.country_name=='unspecified')&(joined_df.type=='export'),'value'].sum(),
 joined_df.loc[(joined_df.country_name=='unspecified')&(joined_df.type=='import'),'value'].sum(),
 joined_df.loc[(joined_df.country_name=='unspecified')&(joined_df.type=='export'),'value'].sum()/joined_df.loc[joined_df.type=='export','value'].sum(),
 joined_df.loc[(joined_df.country_name=='unspecified')&(joined_df.type=='import'),'value'].sum()/joined_df.loc[joined_df.type=='import','value'].sum())

(11374.543572999999, 32985.857874, 0.007810923060833724, 0.01966533343392377)

In [14]:
print(joined_df.shape[0])
joined_df=joined_df.dropna(subset=['country_name'])
joined_df=joined_df.loc[joined_df.country_name!='unspecified',:] #This is because as shown above the percentage of transactions
#where country is unspecified is very small and because we need to show it on a map its better to remove it for demo purposes
print(joined_df.shape[0])

122939
122122


In [15]:
joined_df.describe(include='all')

,pc_description,unit,country_name,quantity,value,type,year
count,122122,116508,122122,5.688900e+04,122122.000000,122122,122122
unique,169,7,246,NaN,NaN,2,4
top,other commodities,na,u s a,NaN,NaN,export,2017_2018
freq,1541,48175,1250,NaN,NaN,83664,31302
mean,NaN,NaN,NaN,5.374857e+06,22.810899,NaN,NaN
std,NaN,NaN,NaN,7.506464e+07,262.854773,NaN,NaN
min,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,NaN
25%,NaN,NaN,NaN,1.450000e+02,0.022087,NaN,NaN
50%,NaN,NaN,NaN,5.672000e+03,0.304389,NaN,NaN
75%,NaN,NaN,NaN,1.682240e+05,3.260038,NaN,NaN


In [16]:
joined_df.groupby(['pc_description','country_name','year','type']).agg('count')['value'].value_counts()

1    122122
Name: value, dtype: int64

In [17]:
joined_df_2=joined_df
#Since pivot method is not accepting multiple values for index
joined_df_2['index_2']=joined_df_2.apply(lambda x:"{}_{}_{}".format(x['pc_description'],x['country_name'],x['year']),axis=1)
joined_df_2.columns,joined_df_2.head()

(Index(['pc_description', 'unit', 'country_name', 'quantity', 'value', 'type',
        'year', 'index_2'],
       dtype='object'),
       pc_description                    ...                                                      index_2
 1  other commodities                    ...                            other commodities_kenya_2014_2015
 2  other commodities                    ...                     other commodities_tanzania rep_2014_2015
 3  other commodities                    ...                           other commodities_uganda_2014_2015
 4  other commodities                    ...                            other commodities_benin_2014_2015
 5  other commodities                    ...                     other commodities_burkina faso_2014_2015
 
 [5 rows x 8 columns])

In [18]:
joined_df_imp_exp_pivot=joined_df_2.pivot(index='index_2',columns='type',values='value')
joined_df_imp_exp_pivot=joined_df_imp_exp_pivot.fillna(0)
joined_df_imp_exp_pivot.head(),joined_df_imp_exp_pivot.shape

(type                                                  export  import
 index_2                                                             
 ac, refrigeration machnry etc_afghanistan_2014_...  1.016735     0.0
 ac, refrigeration machnry etc_afghanistan_2015_...  1.868763     0.0
 ac, refrigeration machnry etc_afghanistan_2016_...  0.450290     0.0
 ac, refrigeration machnry etc_afghanistan_2017_...  0.520000     0.0
 ac, refrigeration machnry etc_albania_2014_2015     0.025633     0.0,
 (88308, 2))

In [19]:
print(joined_df_2.shape[0])
joined_df_3=joined_df_2.groupby('index_2').first()
joined_df_3.head(),joined_df_3.shape

122122


(                                                                   pc_description    ...           year
 index_2                                                                              ...               
 ac, refrigeration machnry etc_afghanistan_2014_...  ac, refrigeration machnry etc    ...      2014_2015
 ac, refrigeration machnry etc_afghanistan_2015_...  ac, refrigeration machnry etc    ...      2015_2016
 ac, refrigeration machnry etc_afghanistan_2016_...  ac, refrigeration machnry etc    ...      2016_2017
 ac, refrigeration machnry etc_afghanistan_2017_...  ac, refrigeration machnry etc    ...      2017_2018
 ac, refrigeration machnry etc_albania_2014_2015     ac, refrigeration machnry etc    ...      2014_2015
 
 [5 rows x 7 columns], (88308, 7))

In [20]:
joined_df_4=joined_df_3.join(joined_df_imp_exp_pivot,how='inner')
joined_df_4.shape,joined_df_4.head(),joined_df_4.columns,joined_df_4.isnull().sum()

((88308, 9),
                                                                    pc_description  ...   import
 index_2                                                                            ...         
 ac, refrigeration machnry etc_afghanistan_2014_...  ac, refrigeration machnry etc  ...      0.0
 ac, refrigeration machnry etc_afghanistan_2015_...  ac, refrigeration machnry etc  ...      0.0
 ac, refrigeration machnry etc_afghanistan_2016_...  ac, refrigeration machnry etc  ...      0.0
 ac, refrigeration machnry etc_afghanistan_2017_...  ac, refrigeration machnry etc  ...      0.0
 ac, refrigeration machnry etc_albania_2014_2015     ac, refrigeration machnry etc  ...      0.0
 
 [5 rows x 9 columns],
 Index(['pc_description', 'unit', 'country_name', 'quantity', 'value', 'type',
        'year', 'export', 'import'],
       dtype='object'),
 pc_description        0
 unit                425
 country_name          0
 quantity          45481
 value                 0
 type             

In [0]:
joined_df_4.unit=joined_df_4.unit.fillna('NA')
joined_df_4.quantity=joined_df_4.quantity.fillna('NA')

In [22]:
joined_df_4=joined_df_4.drop(columns=['value']).rename(columns={'import':'import_amt','export':'export_amt'})
joined_df_4.columns,joined_df_4.shape,joined_df_4.head()

(Index(['pc_description', 'unit', 'country_name', 'quantity', 'type', 'year',
        'export_amt', 'import_amt'],
       dtype='object'),
 (88308, 8),
                                                                    pc_description    ...     import_amt
 index_2                                                                              ...               
 ac, refrigeration machnry etc_afghanistan_2014_...  ac, refrigeration machnry etc    ...            0.0
 ac, refrigeration machnry etc_afghanistan_2015_...  ac, refrigeration machnry etc    ...            0.0
 ac, refrigeration machnry etc_afghanistan_2016_...  ac, refrigeration machnry etc    ...            0.0
 ac, refrigeration machnry etc_afghanistan_2017_...  ac, refrigeration machnry etc    ...            0.0
 ac, refrigeration machnry etc_albania_2014_2015     ac, refrigeration machnry etc    ...            0.0
 
 [5 rows x 8 columns])

In [23]:
joined_df_4.describe(include='all')

,pc_description,unit,country_name,quantity,type,year,export_amt,import_amt
count,88308,88308,88308,88308,88308,88308,88308.000000,88308.000000
unique,169,8,246,23190,2,4,NaN,NaN
top,other commodities,na,u s a,NA,export,2017_2018,NaN,NaN
freq,855,33615,660,45481,83664,22655,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,12.924512,18.620905
std,NaN,NaN,NaN,NaN,NaN,NaN,133.467547,279.179856
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,0.014359,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,0.240000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.503317,0.135696


In [0]:
joined_df_4.to_csv('processed_data_out_pd.csv',index=False)

In [0]:
from google.colab import files

files.download('processed_data_out_pd.csv')